In [ ]:

!pip install piq opencv-python pillow torchvision tqdm --quiet

# Installed version of scikit-image and image-quality to enable the use of the `image_quality` package.
!pip install scikit-image==0.19.3 image-quality


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\OTHNIEL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit-image-0.19.3.tar.gz (22.2 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\OTHNIEL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 106, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 97, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^

In [ ]:
import os
import numpy as np
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from PIL import Image
from piq import brisque 
import torch

In [ ]:


# Mock scoring function 
def process_image(filepath):
    try:
        image = Image.open(filepath).convert("RGB")
        tensor = torch.from_numpy(np.array(image)).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        score = brisque(tensor, data_range=1.0).item()
        return os.path.basename(filepath), score
    except Exception as e:
        print(f"Error processing {filepath}: {e}")
        return os.path.basename(filepath), None



In [ ]:
# --- Configuration ---
input_dir = "imagessss"
output_csv = "image_quality_scores.csv"
checkpoint_interval = 100  # save every 100 images
max_workers = 8  # 8 images at a time



In [ ]:
# --- Main Script ---
files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith((".jpg", ".png"))]
results = []

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(process_image, file): file for file in files}

    with open(output_csv, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["Filename", "BRISQUE Score"])

        for i, future in enumerate(tqdm(as_completed(futures), total=len(futures))):
            filename, score = future.result()
            writer.writerow([filename, round(score, 2) if score is not None else "Error"])
            results.append((filename, score))

            if i % checkpoint_interval == 0 and i != 0:
                csv_file.flush()  # force write to disk

print(f"✅ Done! {len(results)} images processed. Results saved to {output_csv}")


100%|██████████| 20048/20048 [12:20<00:00, 27.07it/s]

✅ Done! 20048 images processed. Results saved to image_quality_scores.csv


In [ ]:
!pip install reportlab

from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import csv

# --- Configuration ---
csv_path = "image_quality_scores.csv"  # Input CSV file
pdf_path = "image_quality_report.pdf"  # Output PDF file
max_rows_per_page = 40

# --- PDF Setup ---
styles = getSampleStyleSheet()
doc = SimpleDocTemplate(pdf_path, pagesize=A4)
elements = []

# Adding Title and Subheading
title = Paragraph("🖼️ Image Quality Assessment Report", styles['Title'])
subtitle = Paragraph("Generated using BRISQUE Metric", styles['Heading2'])
elements += [title, subtitle, Spacer(1, 12)]

# --- Load CSV and build table chunks ---
with open(csv_path, newline='') as f:
    reader = list(csv.reader(f))
    headers = reader[0]
    data_rows = reader[1:]

    for i in range(0, len(data_rows), max_rows_per_page):
        chunk = data_rows[i:i + max_rows_per_page]
        table_data = [headers] + chunk

        table = Table(table_data, colWidths=[250, 150])
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#d0d0ff")),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
            ('ALIGN', (1, 1), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
            ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 10),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.whitesmoke, colors.lightgrey]),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ]))

        elements.append(table)
        if i + max_rows_per_page < len(data_rows):
            elements.append(PageBreak())

# --- Generate PDF ---
doc.build(elements)
print(f"✅ PDF report generated: {pdf_path}")



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\OTHNIEL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


Exception ignored in: <function tqdm.__del__ at 0x000002273FA01DA0>
Traceback (most recent call last):
  File "C:\Users\OTHNIEL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "C:\Users\OTHNIEL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


✅ PDF report generated: image_quality_report.pdf
